## 11.1 Retrofit - podstawy

Przechodzimy do serii aplikacji w których zapoznamy się z biblioteką `Retrofit` - jedną z najpopularniejszych bibliotek służąca wysyłaniu i odbieraniu danych z zewnętrznych serwisów. Generuje ona kod sieciowy - z naszej strony musimy zadeklarować interfejs wraz z metodami opisującymi operacje sieciowe które chcemy wykonać. Analogicznie jak w przypadku biblioteki `ROOM` będziemy posługiwać się adnotwacjami w celu dostarczenia informacji na podstawie których kod zostanie wygenerowany przez `Retrofit`.

W tej części wykonamy `GET` aby pobrać dane z serwera i wyświetlić je w naszej aplikacji. Aby maksymalnie uprościć elementy niezwiązane z `Retrofit` posłużymy się pojedynczą aktywnością oraz jednym polem `TextView`.

Nasza aplikacja będzie komunikować się z zewnętrznym serwisem, więc niezbędne jest odpowiednie upoważnienie

In [ ]:
<uses-permission android:name="android.permission.INTERNET"/>

Następnie musimy dodać `Retrofit` do zależności w naszej aplikacji

In [ ]:
implementation 'com.squareup.retrofit2:retrofit:2.9.0'

Aby przekonwertować odpowiedź `Response` serwera na odpowiednie obiekty w Javie będziemy potrzebować odpowiedni konwerter - dostępnych jest [kilka](https://square.github.io/retrofit/), nas interesuje `Gson`

In [ ]:
implementation 'com.squareup.retrofit2:converter-gson:2.9.0'

W tym przykładzie wykorzystamy [**JSONPlaceholder**](https://jsonplaceholder.typicode.com/), który jest API przeznaczonym do testowania. Mamy dostępnych kilka **endpointów**
- posts
- comments
- albums
- photos
- todos
- users

W tym przykładzie wybierzemy pierwszy (posts), rozpoczniemy komunikację z tym serwerem oraz asynchronicznie wykonamy operację `GET` - czyli pobierzemy wszystkie posty i wrzucimy jch treść do `TextView`. Posty znajdziemy w formacie `JSON` - jest to format służący komunikacji pomiędzy naszą aplikacją a serwerem. Na serwerze znajduje się w chwili pisania 100 postów, struktura pojedynczego postu wygląda następująco

In [ ]:
{
    "userId": 1,
    "id": 1,
    "title": "sunt aut ...",
    "body": "quia et ..."
},

W pierwszym kroku musimy stworzyć nasz model danych odpowiadający strukturze obecnej na serwerze. Więc do klasy `Post` dodajemy pola `userId`, `id`, `title` oraz `body`. Jeżeli chcemy wykorzystać inną nazwę musimy użyć adnotacji `@SerializedName` - tutaj zmienimy nazwę `body` na `content`. W argumencie `@SerializedName` podajemy nazwę którą chcemy zmienić jako `String` - czyli ta nazwa musi odpowiadać nazwie obecnej w formacie dostępnym na serwerze.

In [ ]:
data class Post (
    val userId: Int,
    val id: Int,
    val title: String,

    @SerializedName("body")
    val content: String
)

Zmieńmy jeszcze layout gfłównej aktywności - chcemy wyświetlić wszystkie posty w jednym polu `TextView`, więc umieścimy je w `NestedScrollView` aby umożliwić przewijanie.

In [ ]:
<androidx.constraintlayout.widget.ConstraintLayout 
    xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".MainActivity">

    <androidx.core.widget.NestedScrollView
        android:layout_width="match_parent"
        android:layout_height="match_parent">

        <TextView
            android:id="@+id/textView"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:text="Hello World!"
            app:layout_constraintBottom_toBottomOf="parent"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toTopOf="parent" />

    </androidx.core.widget.NestedScrollView>

</androidx.constraintlayout.widget.ConstraintLayout>

Teraz musimy stworzyć interfejs w którym zdefiniujemy metodę służącą zwróceniu danych z serwera

In [ ]:
interface PlaceholderApi {
    fun posts(): Call<List<Post>>
}

Zwracanym obiektem jest `Call` zawierający listę wszystkich postów. Oprócz danych w tym obiekcie znajdują się również obiekty `Response` oraz `Request` zawierające informacje o samym połączeniu oraz metoda `enqueue` pozwalająca na asynchroniczne wysłanie żądania oraz powiadomienia zwrotnego, lub błędu - innymi słowy `Call` hermetyzuje pojedynczy `Request` oraz pojedynczy `Response`.

Musimy wykorzystać adnotację `@GET` aby poinformować `Retrofit` co dokładnie ta metoda ma robić. Dzięki temu `Retrofit` będzie w stanie wygenerować odpowiedni kod.

In [ ]:
interface PlaceholderApi {
    @GET("posts")
    fun posts(): Call<List<Post>>
}

Adres URL naszego API wygląda następująco [https://jsonplaceholder.typicode.com/posts](https://jsonplaceholder.typicode.com/posts). Część "https://jsonplaceholder.typicode.com" jest adresem bazowym, "posts" jest relatywnym adresem (endpoint) - przy adnotacjach w interfejsie posługujemy się tylko relatywnym adresem.

Przejdźmy do głównej aktywności - w metodzie `onCreate` wykonamy nasze żądanie `GET`. W pierwszej kolejności potrzebujemy instancję `Restrofit`

In [ ]:
val retrofit = Retrofit.Builder()

Następnie podajemy `baseUrl` - jak widzieliśmy wqcześniej będzie to "https://jsonplaceholder.typicode.com"

In [ ]:
        .baseUrl("https://jsonplaceholder.typicode.com/")

Definiujemy nasz `ConverterFactory` - my używamy formatu `JSON`, więc konwerterem będzie `Gson`

In [ ]:
        .addConverterFactory(GsonConverterFactory.create())

na koniec wywołujemy metodę `build`

In [ ]:
val retrofit = Retrofit.Builder()
    .baseUrl("https://jsonplaceholder.typicode.com/")
    .addConverterFactory(GsonConverterFactory.create())
    .build()

Posiadając instancję `Retrofit` możemy utworzyć instancję `PlaceholderApi`.

In [ ]:
val api = retrofit.create(PlaceholderApi::class.java)

Aby wykonać żądanie sieciowe wymagany jest obiekt typu `Call`

In [ ]:
val posts = getData(api)

Na tym obiekcie możemy wykonać `execute` - jest to wykonanie synchroniczne. Jeżeli nastąpi próba wykonania operacji sieciowych na wątku głównym zostanie rzucony wyjątek. Musimy wykonać operację sieciową asynchronicznie - tutaj `Retrofit` dostarcza odpowiednią metodę `enqueue`, której parametrem jest obiekt o typie interfejsu `Callback`. 

Zaznaczmy że w kotlinie nie wykorzystuje się zazwyczaj tej metody - zamiast niej korzystamy z `Coroutines`. Wersja z `Coroutines` znajduje się poniżej.

In [ ]:
call.enqueue(object : Callback<List<Post>> {
    override fun onResponse(
        call: Call<List<Post>?>, 
        response: Response<List<Post>?>) {
    }

    override fun onFailure(call: Call<List<Post>?>, t: Throwable) {
    }
})

Musimy zaimplementować dwie metody
- `onResponse` - wykonywana przy sukcesie komunikacji z serwerem - co oznacza samą komunikację a nie powodzenie samej operacji (przykładowo możemy dostać znany kod 404 przy próbie dostępu do danych które nie istnieją na serwerze)
- `onFailure` - wykonywana przy braku komunikacji z serwerem

W metodzie `onResponse` mamy dwa argumenty reprezentujące obiekty `Call` i `Response`. W pierwszym kroku implementacji tej metody sprawdzamy czy odpowiedź jest poprawna

In [ ]:
if (response.isSuccessful) {

Czyli kod który otrzymujemy mieści się w zakresie 200 - 300 - więcej informacji o kodach [tutaj](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status). Następnie musimy rozpakować dane - są one przechowywane w polu `body` obiektu `Response`. Oprócz tego `Response` posiada jeszcze pole typu `okhttp3.Response` zawierające kod odpowiedzi, rodzaj wykorzystanego protokołu (`HTTP 1.1`), oraz kilka innych informacji - w tym przykładzie są one nie istotne.

In [ ]:
val posts = response.body()

Następnie, wykorzystując pętlę `foreach`, umieścimy wszystkie dane w polu `TextView`

In [ ]:
posts?.forEach {
   val content = StringBuilder()
   content.append("id: ").append(it.id).append("\n")
       .append("UserId: ").append(it.userId).append("\n")
       .append("title: ").append(it.title).append("\n")
       .append("body: ").append(it.title).append("\n\n")
   textView.append(content)

Jeżeli dostaniemy odpowiedź z kodem spoza podanego wyżej zakresu, drukujemy w polu `TextView` ten kod 

In [ ]:
} else {
    textView.text = "Code: ${response.code()}"
}

Metoda `onFailure` posiada argument `Throwable` - jest to superklasa `Exception` i `Error`. Tutaj wydrukujemy wiadomość która zostanie wysłana przy niepowodzeniu operacji

In [ ]:
override fun onFailure(
    call: Call<List<Post>?>,
    t: Throwable) {
   textView.text = t.message
}

Możemy przetestować aplikację - samo ściągnięcie danych i wyświetlenie może zająć kilkanaście sekund

<img src="https://media2.giphy.com/media/TjtDMwfxyC6HEIh29i/giphy.gif?cid=790b761100981bb41312fc0d0d08dd7a7d2672b1e408e844&rid=giphy.gif&ct=g" width="150" />

Zamiast metody `enqueue` klasy `Call` z biblioteki `Retrofit`, teraz wykorzystamy `Coroutines`. W naszym interfejsie możemy zostać przy obiekcie typu `Call`, lecz często spotykanym jest odebranie obiektu typu `Response` - jednak musimy zamienić `getData` na `suspend fun`.

W głównej aktynwości dodajmy metodę pomocniczą, która asynchronicznie wyciągnie odpowiedź serwera na wątku roboczym.

In [ ]:
private suspend fun getData(api: PlaceholderApi): List<Post> {
   return withContext(Dispatchers.IO) {
       val response = async { api.posts() }
       val result = response.await().body()
       return@withContext result!!
   }
}

Przeanalizujmy składnię. `withContext` rozpoczyna wykonanie i zawiesza je do czasu otrzymania wartości zwracanej.

In [ ]:
val response = async { api.posts() }

`async` rozpoczyna nową `Courutine` wykonywaną asynchronicznie i zwracającą wartość zapisaną w `response` typu `Deferred`.

In [ ]:
val result = response.await().body()

Funkcja `await` blokuje aktualną `Coroutine` dopóki `response` nie jest dostępny.

Następnie chcemy zwrócić listę wszystkich `Post`, więc w pierwszym kroku musimy zwrócić z bloku `withContext`

In [ ]:
return@withContext result!!

Ponieważ `withContext` również zwraca wartość, więc możemy umieścić przed nim `return`.

In [ ]:
return withContext(Dispatchers.IO) {

Tutaj ważna rzecz - elementy ui możemy modyfikować tylko na **wątku głównym**, więc w metodzie `onCreate` aktualizację ui wykonujemy na wątku głównym.

In [ ]:
CoroutineScope(Dispatchers.Main).launch {
    val posts = getData(api)
    posts.forEach {
       val content = StringBuilder()
       content.append("id: ").append(it.id).append("\n")
           .append("UserId: ").append(it.userId).append("\n")
           .append("title: ").append(it.title).append("\n")
           .append("body: ").append(it.title).append("\n\n")
       textView.append(content)
   }
}

Całość możemy wykonać w jednym bloku w metodzie `onCreate`

In [ ]:
GlobalScope.launch(Dispatchers.Main) {

    val posts = withContext(Dispatchers.IO) {
        val response = async { api.posts() }
        return@withContext response.await()
    }
    val call = posts.await()
    call.forEach {
        val content = StringBuilder()
        content.append("id: ").append(it.id).append("\n")
            .append("UserId: ").append(it.userId).append("\n")
            .append("title: ").append(it.title).append("\n")
            .append("body: ").append(it.title).append("\n\n")
        textView.append(content)
    }
}